# 統計の基礎
統計は、標本を調べることで母集団の性質を明らかにすることを目的として、個々の要素を標本化してこれを分析し、母集団の性質を数量的に表すことである
## 統計とは
母集団の特徴を表すのが母数θ  
母数は母平均μ、母分散σ^2などのいくつかの候補がある  
母集団すべてを知ることができない場合には、これから要素をいくつか抽出し、これを標本化という  
標本から計算される数値を統計量といい、これを通して母数を推定する

## 推定
標本から得られた統計量をもとに、母数の存在する範囲を求めることを統計的推定という  
推定には1つの値を推定する点推定と、範囲を推定する区間推定がある

### 点推定
不偏性とは、推定量の期待値が母数に一致することである  
標本平均の分布を見てみると、与えた平均値の周りに分布しているようすが認められる

In [1]:
from scipy.stats import norm
import numpy as np
import matplotlib.pyplot as plt

num = 50
N = 10
mean, std = 2, 0.5 # それぞれ平均値と標準偏差
mu = np.zeros(num)
for i in range(num) :
    mu[i] = np.mean(norm.rvs(loc=mean, scale=std, size=N))
plt.hist(mu, bins=20, range=(1.5,2.5))

(array([0., 0., 0., 0., 1., 3., 4., 3., 8., 7., 3., 9., 4., 5., 1., 2., 0.,
        0., 0., 0.]),
 array([1.5 , 1.55, 1.6 , 1.65, 1.7 , 1.75, 1.8 , 1.85, 1.9 , 1.95, 2.  ,
        2.05, 2.1 , 2.15, 2.2 , 2.25, 2.3 , 2.35, 2.4 , 2.45, 2.5 ]),
 <a list of 20 Patch objects>)

#### 標本分散の自由度
自由度とは、いくつかの変数を勝手にその値を動かしてもよいかを測る指標である

#### 一致性
標本数が増えるほど、推定量がだんだんと母数に近づく性質を一致性という。  
このような性質を持つ推定量のことを一致推定量という。

### 区間推定
標本平均や標本分散は母平均や母分散の周りに分布する。これらがどれくらいの確率で、その分布のどれほどの区間に収まるのか推定することを区間推定という。  
これの指標の一つとして信頼区間がある。信頼区間は、(1-α)の確率で真の母数の値θが区間[L,U]に入る区間の事を言う。  
区間[L,U]をパーセント点の表記[-Zα/2,Zα/2]に置き換えて考える。  
1-α=0.9を求める場合は以下のようにする。

In [5]:
alp = 0.01
za, zb = norm.interval(alpha=(1-alp), loc=0, scale=1)
print('za = ', za, ' zb =  ', zb)

za =  -2.5758293035489004  zb =   2.5758293035489004


### 母平均の信頼区間
母分散が既知の場合と、母分散が未知の場合で信頼区間の求め方は異なる。  
詳しい式は省く。  
両方の求めかたに共通して言えることとして、
- 信頼度を高くすれば信頼区間が広がる。逆もまた然りである。
- 高すぎる信頼度はあまり意味をなさない。信頼度100%とは、推定する母平均がどんな値でもはずれないということだからだ
- 標本数を増やせば、その分、母平均の値の絞り込みができることになる

**例題**  
小学校のある学年の全国児童数は110万人とする。この児童への全国テストの平均値を推定するため、N=10人を無作為抽出して、これに対する標本平均は145.2だった。また、標本標準偏差は23.7点だった。このとき信頼度が0.99,0.95,0.90に対する信頼区間を求めよう。

In [7]:
from scipy.stats import t
N = 10 # 標本数
mu_hat = 145.2 # 標本平均
std_hat = 23.7 # 標本標準偏差
t1 = t.interval(0.99, df=N-1)
t2 = t.interval(0.95, df=N-1)
t3 = t.interval(0.90, df=N-1)
se = std_hat / np.sqrt(N)

print('0.99, interval : ', mu_hat+t1[0] * se, mu_hat+t1[1] * se)
print('0.95, interval : ', mu_hat+t2[0] * se, mu_hat+t2[1] * se)
print('0.90, interval : ', mu_hat+t3[0] * se, mu_hat+t3[1] * se)

0.99, interval :  120.84378885405553 169.55621114594445
0.95, interval :  128.24604132892404 162.15395867107594
0.90, interval :  131.46155538107368 158.9384446189263


### 母比率の信頼区間
母比率は、内閣支持率やTV視聴率などを表す。
詳しい式は省く。

**例題**  
内閣支持率を調べるために、世論調査を行った。サンプル数は1000人で、支持する人の数は550人であった。信頼度95%の信頼区間を求めよう。

In [11]:
from scipy.stats import binom
N = 1000 # 標本数
alp = 0.95 # 信頼度
p = 550 / N # 標本比率

bottom, up = binom.interval(alpha=alp, n=N, p=p, loc=0)
print('0.95の信頼区間 : {} < p < {}'.format(bottom/N, up/N))

0.95の信頼区間 : 0.519 < p < 0.581


## 仮説検定
仮説検定とは、ある仮説に対して、それが正しいのか否かを統計学的に検証する手段である。  
検証を行うため立てる仮説のことを帰無仮説といい、それに対する仮説のことを対立仮説という。  
有意水準とは、仮説検定を行う場合に、帰無仮説を棄却するかどうかを判定する基準を言う。

### 片側検定と両側検定
帰無仮説は「平均μ=2.0である」というように、等号の形で設定されることが多い。これに対する対立仮説を「μ > 2.0」,「μ < 2.0」とおくことを片側検定という。「μ≠2.0」とおくことを両側検定という。


### 母平均の検定
2通りの方法がある。
- 母分散が既知の場合 → z検定を使用する
- 母分散が未知の場合 → t検定を使用する

**例題(片側検定)**  
あるクラスでは、数学の平均点を上げるべく補講の前後でテストを行った。補講前後の点数差は,1,-1,-2,3,-1,5,4,0,7,-1であった。補講の効用を知りたい。有意水準α=5%で検定しよう。  
  
補講の効用がなかったことを帰無仮説に、あったことを対立仮説にする。

In [14]:
from scipy import  stats

data = np.array([1,-1,-2,3,-1,5,4,0,7,-1])
m = np.average(data) # 標本平均
s = np.std(data, ddof=1) # 標本標準偏差(自由度をN-1にしている)
N = len(data) # 標本数

alp = 0.05
talp = stats.t.ppf((1-alp), N-1) # αに対応する確率変数の値
print('talp (alpha=0.05, df=%d) =%f' %((N-1), talp))

m0 = 0
t = (m-m0) / (s/np.sqrt(N))
print('t = ', t)

talp (alpha=0.05, df=9) =1.833113
t =  1.5480470613460082


talp>tのため、帰無仮説を棄却できない。このように確率変数の値で比較するのではなく、確率の値で比較するには、以下のようにする。

In [15]:
prob = stats.t.cdf(t, N-1)
print('p = ', 1-prob)

p =  0.07800883831234118


pが設定した有意水準より大きいため、帰無仮説を棄却できない。

**例題2(両側検定)**  
アルファ社の精密部品の直径の規格は1.54cmである。製造したものから8個をサンプリングし、直径を測ると1.5399, 1.5390, 1.5399, 1.5395, 1.5400, 1.5390, 1.5399, 1.5399であった。この部品は規格どおりであるか？有意水準α=5%で検定しよう。

In [18]:
data2 = np.array([1.5399, 1.5390, 1.5399, 1.5395, 1.5400, 1.5390, 1.5399, 1.5399])
m = np.mean(data2)
s = np.std(data2, ddof=1)
N = len(data2)
df = N-1
m0 = 1.54

t = (m-m0) / (s/np.sqrt(N))
prob = stats.t.cdf(t, df)
if t >= 0 :
    p = 1-prb
else :
    p = prob

print('t = ', t)
print('p = ', 2*p) # 両側検定のため、2*pにしている

t =  -2.4373067467182707
p =  0.04493615922381196


### 母分散の検定
標本が正規分布に従っていると仮定する。こ正規分布の平均と分散が未知の時、母分散が仮定する推定量に等しいか否かを判定するのが母分散の検定である。

**例題**  
先のアルファ社の精密部品について、規格では分散は1*10^-7mm以下としている(ばらつきが小さいことを言いたい)。精密部品の簿標準偏差は、この規格からずれているかどうか、有意水準5%で検定しよう。  
  
分散がある値以下であるかを問いただしているため、片側検定である。そのため、帰無仮説を`分散=1*10^-7`、対立仮説を`分散>1*10^-7`とする。

In [21]:
var = np.var(data2, ddof=1) # 標本分散
print('分散 : ', var)

var0 = 1.e-7
alp = 0.05
N = len(data2)
df = N-1

chi2 = (N-1) * var / var0
chi2_alp = stats.chi2.ppf((1-alp), df=df)
print('chi2 = ', chi2, ' chi2_alp = ', chi2_alp)
pval = 1-stats.chi2.cdf(chi2, df)
print('pval = ', pval)

分散 :  1.769642857143276e-07
chi2 =  12.387500000002934  chi2_alp =  14.067140449340169
pval =  0.08851446668234408


### 2標本の平均の差の検定
2つの標本の母集団の母平均の差を検定することを考える。すなわち、2つの母平均は同じか異なるかをしりたいとする。この場合、次の3通りが考えられる。
- 2角母分散が既知
- 2つの母分散は未知であるが、この2つは等しい
- 2つの母分散がともに未知である　→　ウェルチのt検定

**例題 2つの体温計の性能検定**  
次は、2つの体温計(s1)と(s2)の測定精度に差があるかを確かめた10回の測定結果である。この2つの体温計は同じような精度であるといえるだろうか？  

両方の平均値の差に有意差があるか否かの検定を行う。このため、帰無仮説をμx = μy(標本の)とし、対立仮説をμx ≠μy(標本の)とする。このため、両側検定になる。

In [23]:
s1 = np.array([37.1,36.7,36.6,37.4,36.8,36.7,36.9,37.4,36.6,36.7])
s2 = np.array([36.8,36.6,36.5,37.0,36.7,36.5,36.6,37.1,36.4,36.7])

t, p = stats.ttest_ind(s1, s2, equal_var= False)
print('t = ', t, ' p = ', p)

t =  1.6653821449601343  p =  0.11477658092297612


有意水準を5%とすると、pがこれより大きいため、帰無仮説は棄却できない。すなわち2つの体温計の平均値が等しいという仮説は棄却できない。

### 相関, 無相関の検定
相関の基本的な説明は省く。
相関があるか否かを検定する、無相関の検定がある。

**例題**  
表4.2(このノートブックでは、以下のスクリプトの変数を参照)は父の身長x、とその息子の身長yであり、この両者の間に相関があるかどうか(親の身長が高ければ、その子供の身長も高い、という仮説)を有意水準α=0.05, 0.01の場合について調べよう。  
  
この問題の場合、帰無仮説は相関なし、対立仮説は相関ありである。

In [24]:
x = np.array([168,172,181,179,166,185,177,176,169,161])
y = np.array([111,125,129,120,126,133,130,116,118,115])

corr, p = stats.pearsonr(x,y)
print('corr = ', corr, ' p = ', p)

corr =  0.6342703173343618  p =  0.04888299019331427


有意水準α=0.05のときは棄却し、α=0.01のときは、棄却しないという結果になった。